In [33]:
%reload_ext autoreload
%autoreload 2

from importlib import import_module
from pprint import pprint
from deep_image_matching import logger, timer
from deep_image_matching.config import Config
from deep_image_matching.image_matching import ImageMatching
from deep_image_matching.io.h5_to_db import export_to_colmap


Get the list of possible configurations and chose one of them.

In [5]:
Config.get_config_names()

['superpoint+lightglue',
 'superpoint+lightglue_fast',
 'superpoint+superglue',
 'disk+lightglue',
 'aliked+lightglue',
 'orb+kornia_matcher',
 'sift+kornia_matcher',
 'loftr',
 'se2loftr',
 'roma',
 'keynetaffnethardnet+kornia_matcher',
 'dedode']

Build a dictionary with the input processing parameters (they are the same as the input parameters for the CLI and GUI) and pass it to the Config class to get the initialize the configuration object.
Refer to the README for more information about the parameters.

In [29]:
cli_params = {
    "dir": "/home/francesco/casalbagliano/subset_B",
    "config": "superpoint+superglue",
    "strategy": "matching_lowres",
    "quality": "high",
    "tiling": "preselection",
    "skip_reconstruction": True,
    "force": True,
    "verbose": True,
}
cfg = Config(cli_params)

2024-01-07 16:43:07 | [WARNING ] /home/francesco/casalbagliano/subset_B/results_superpoint+superglue_matching_lowres_quality_high already exists, removing /home/francesco/casalbagliano/subset_B/results_superpoint+superglue_matching_lowres_quality_high


Check the configuration object and, if you kow what you are doing, you can modify it for updating the configuration.


In [31]:
print("Config general:")
pprint(cfg.general)
print("Config extractor:")
pprint(cfg.extractor)
print("Config matcher:")
pprint(cfg.matcher)

Config general:
{'geom_verification': <GeometricVerification.PYDEGENSAC: 1>,
 'gv_confidence': 0.999999,
 'gv_threshold': 2,
 'image_dir': PosixPath('/home/francesco/casalbagliano/subset_B/images'),
 'matching_strategy': 'matching_lowres',
 'output_dir': PosixPath('/home/francesco/casalbagliano/subset_B/results_superpoint+superglue_matching_lowres_quality_high'),
 'overlap': None,
 'pair_file': PosixPath('/home/francesco/casalbagliano/subset_B/results_superpoint+superglue_matching_lowres_quality_high/pairs.txt'),
 'preselection_size_max': 2000,
 'quality': <Quality.HIGH: 3>,
 'retrieval': None,
 'skip_reconstruction': True,
 'tile_overlap': 50,
 'tile_selection': <TileSelection.PRESELECTION: 3>,
 'tile_size': (2400, 2000),
 'upright': False,
 'verbose': True}
Config extractor:
{'keypoint_threshold': 0.005, 'max_keypoints': 4096, 'name': 'superpoint'}
Config matcher:
{'match_threshold': 0.3, 'name': 'superglue'}


For simplicity, save some of the configuration parameters in variables.

In [32]:
imgs_dir = cfg.general["image_dir"]
output_dir = cfg.general["output_dir"]
matching_strategy = cfg.general["matching_strategy"]
retrieval_option = cfg.general["retrieval"]
pair_file = cfg.general["pair_file"]
overlap = cfg.general["overlap"]
upright = cfg.general["upright"]
extractor = cfg.extractor["name"]
matcher = cfg.matcher["name"]

Initialize the ImageMatching class that will be used for performing the image matching.

In [34]:
img_matching = ImageMatching(
    imgs_dir=imgs_dir,
    output_dir=output_dir,
    matching_strategy=matching_strategy,
    retrieval_option=retrieval_option,
    local_features=extractor,
    matching_method=matcher,
    pair_file=pair_file,
    custom_config=cfg.as_dict(),
    overlap=overlap,
)

2024-01-07 16:45:51 | [DEBUG   ] Matching options: Quality: MEDIUM - Tiling: NONE
2024-01-07 16:45:51 | [DEBUG   ] Saving directory: output
2024-01-07 16:45:51 | [DEBUG   ] Running inference on device cuda
Loaded SuperPoint model
2024-01-07 16:45:52 | [DEBUG   ] Matching options: Tiling: NONE
2024-01-07 16:45:52 | [DEBUG   ] Saving directory: output
2024-01-07 16:45:52 | [DEBUG   ] Running inference on device cuda
Loaded SuperGlue model ("indoor" weights)
2024-01-07 16:45:52 | [INFO    ] Running image matching with the following configuration:
2024-01-07 16:45:52 | [INFO    ]   Image folder: /home/francesco/casalbagliano/subset_B/images
2024-01-07 16:45:52 | [INFO    ]   Output folder: /home/francesco/casalbagliano/subset_B/results_superpoint+superglue_matching_lowres_quality_high
2024-01-07 16:45:52 | [INFO    ]   Number of images: 63
2024-01-07 16:45:52 | [INFO    ]   Matching strategy: matching_lowres
2024-01-07 16:45:52 | [INFO    ]   Image quality: Quality.MEDIUM
2024-01-07 16:45:

Generate pairs to be matched

In [35]:
pair_path = img_matching.generate_pairs()
timer.update("generate_pairs")

2024-01-07 16:46:55 | [INFO    ] Low resolution matching, generating pairs ..
2024-01-07 16:46:55 | [INFO    ] Extracting features from downsampled images...


100%|██████████| 63/63 [00:19<00:00,  3.20it/s]

2024-01-07 16:47:15 | [INFO    ] Matching downsampled images...



100%|██████████| 1953/1953 [00:51<00:00, 37.62it/s]

2024-01-07 16:48:07 | [DEBUG   ] [Timer] | [low-res pair generation] extraction=0.287, geometric verification=0.041, matching=0.002, Total execution=71.624
2024-01-07 16:48:07 | [INFO    ] Found 1396 pairs.


In [36]:
# Try to rotate images so they will be all "upright", useful for deep-learning approaches that usually are not rotation invariant
if upright:
    img_matching.rotate_upright_images()
    timer.update("rotate_upright_images")

In [ ]:
# Extract features
feature_path = img_matching.extract_features()
timer.update("extract_features")

In [ ]:
# Matching
match_path = img_matching.match_pairs(feature_path)
timer.update("matching")

In [ ]:
# Features are extracted on "upright" images, this function report back images on their original orientation
if upright:
    img_matching.rotate_back_features(feature_path)
    timer.update("rotate_back_features")

In [ ]:
# Export in colmap format
database_path = output_dir / "database.db"
export_to_colmap(
    img_dir=imgs_dir,
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_model="simple-radial",
    single_camera=True,
)
timer.update("export_to_colmap")

In [37]:
timer.print()

2024-01-07 16:51:09 | [INFO    ] [Timer] | [Timer] generate_pairs=2237.797, Total execution=2419.879
